In [1]:
from examples.offline_RL_workshop.offline_policies.policy_registry import PolicyType
from examples.offline_RL_workshop.offline_trainings.restore_policy_model import restore_trained_offline_policy
from examples.offline_RL_workshop.custom_envs.custom_envs_registration import register_grid_envs
import gymnasium as gym
from examples.offline.utils import load_buffer_minari
from examples.offline_RL_workshop.visualizations.utils import get_state_action_data_and_policy_grid_distributions
from examples.offline_RL_workshop.utils import state_action_histogram, compare_state_action_histograms
import os
import warnings
from tqdm import tqdm
from examples.offline_RL_workshop.custom_envs.custom_envs_registration import RenderMode, CustomEnv
from examples.offline_RL_workshop.behavior_policies.visualize_custom_policies import render_custom_policy_simple_grid
from examples.offline_RL_workshop.behavior_policies.behavior_policy_registry import BehaviorPolicyType

os.environ["LD_LIBRARY_PATH"] = "/home/ivan/.mujoco/mujoco210/bin"
os.environ["MINARI_DATASETS_PATH"] = "/home/ivan/Documents/GIT_PROJECTS/Tianshou/tianshou/offline_data"
warnings.filterwarnings("ignore", category=UserWarning, module="gymnasium")


/home/ivan/anaconda3/envs/tianshou_dev/lib/python3.8/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment GymV26Environment-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/ivan/anaconda3/envs/tianshou_dev/lib/python3.8/site-packages/google/rpc/__init__.py:20: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.rpc')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  pkg_resources.declare_namespace(__name__)
/home/ivan/anaconda3/envs/tianshou_dev/lib/python3.8/site-packages/pkg_resources/__init__.py:2350: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`

Introduction:
---

Problem definition: ToDo
Examples of applicability:

In summary, the process is simple: 

Phase A - Collect data set, $D$, of state-action paird through a behavior policy (also known as expert policy) $\pi_b$: e.g. a robot randomly moving (or human controlled) in a given space, data collected from an autonomous vehicle, etc. The data collected doesn't need to come from an expert (typically the case in real situations) and during this phase we are not worry in general about a specific task (i.e. rewards). In fact it could be that the data is collected from a robot doing a different task that the one we are interested in. We want just a set of allowed state-action pairs that could be usable for the task in mind.

Phase B - In this phase we want to solve a given task (so we need to design rewards) only through the provided initial data without any interaction with the environment but still be able to find an optimal or near-optimal policy.

Let's see phase A and B with a simple [example](https://docs.google.com/presentation/d/1-cfO7MNcH6iyN4EwyFjI9F3cVu4hYn5ihCqmZHBeeSk/edit#slide=id.g28654ea4ec6_0_8)

Phase A:

1 - Generate MINARI Dataset
2 - CREATE BEHAVIOR POLICY
3 - LOOK AT DATA DISTRIBUTION

In [3]:
ENV_NAME = CustomEnv.Grid_2D_5x5_discrete_V1 #"Grid_2D_5x5_discrete-V1"
DATA_SET_NAME = "Grid_2D_5x5_discrete-V1_data" #ENV_NAME + "_data"
NUM_STEPS = 1000
RENDER_MODE = RenderMode.RGB_ARRAY_LIST
BEHAVIOR_POLICY =  BehaviorPolicyType.random


#Create env
register_grid_envs()
env = gym.make(ENV_NAME, render_mode=RENDER_MODE)

render_custom_policy_simple_grid(
    env_name=ENV_NAME,
    render_mode=RENDER_MODE,
    behavior_policy=BEHAVIOR_POLICY,
    num_steps=NUM_STEPS,
)


#Load data
#data = load_buffer_minari(DATA_SET_NAME)
#print(f"number of elements: {len(data)}")

#state_action_count_data, _ = get_state_action_data_and_policy_grid_distributions(data, env, policy=None, num_episodes=NUM_EPISODES)
#state_action_histogram(state_action_count_data)

#state_action_histogram(state_action_count_policy)
#compare_state_action_histograms(state_action_count_data, state_action_count_policy)



TypeError: <lambda>() takes 1 positional argument but 2 were given

Why is offline RL a difficult problem?


1 - **Not possible to improve exploration**: As the learning algorithm must rely entirely on the static
dataset $D$, there is no possibility of improving exploration: if D does not contain transitions that illustrate high-reward regions for our task it may be impossible to discover those regions. If we explore beyond our data we could have severe problems as there could be a good reason why this data is not in our dataset: maybe there is an obstacle that could damage the robot or a fragile object that could be damaged by the robot!

Note that this is opposite to online RL where you explore by interacting with the environment. 

This is why the collecting data phase is so important!!

2 - **Distributional shift**: state-action pair distribution in $D$ does not accurately represent the distribution of states-actions of the trained policy. This challenges many existing machine learning methods, which assume that data is independent and identically distributed (i.i.d.). In standard supervised learning, we aim to train a model that performs well on data from the same distribution as the training data. In offline RL, our goal is to learn a policy that behaves differently (hopefully better) than what's seen in the dataset $D$. As a consequence (see later) the RL algorithms will tend to generate actions not included in $D$ and so generate **out of distribution actions data**. This could be dangerous as during inference these actions could bring the system to unexplored states (i.e. not included in $D$).

ToDo: See later some example about it. It could be a 2D grid where the data was collected considering an obstacle but in inference we use the same grid without obstacles and we start from an obstacle zone??? ).

In [ ]:
!pdfexport presentation.pdf